# Playground:  Segmentation workflow for centrosomes

This notebook contains an optimized workflow for segmenting centrosome imaging data. It is based on the Allen Institute workflow for "dots-like" structures, including Centrin-2, Desmoplakin, and PMP34.

Key steps of the workflows:

* Min-max intensity normalization
* 2D Gaussian smoothing (slice-by-slice)
* 3D spot filter to detect dots
* Watershed for seperating falsely merged dots

In [ ]:
import numpy as np
import os

# package for 3d visualization
from itkwidgets import view                              
from aicssegmentation.core.visual import seg_fluo_side_by_side,  single_fluorescent_view, segmentation_quick_view
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 12]

# package for io 
from aicsimageio import AICSImage                            
import imageio

# function for core algorithm
from aicssegmentation.core.seg_dot import dot_3d, dot_3d_wrapper 
from aicssegmentation.core.pre_processing_utils import intensity_normalization, image_smoothing_gaussian_slice_by_slice
from skimage.morphology import remove_small_objects, dilation, erosion, ball     # function for post-processing (size filter)


## Loading the data

We'll start by investigating the segmentation for a single image. You can change which image you're investigating using the FILE_NAME variable below.


In [ ]:
# Update the path to your data
# For those on a Mac system, you can update your username


FILE_PATH = '/data/centrosomes/raw-data/'

FILE_NAME = 'NC12_interphase_Slide22_Emb21_Img1.tif'

reader = AICSImage(FILE_PATH + FILE_NAME) 
IMG = reader.data


## Preview of the image

In [ ]:
#####################
structure_channel = 0
#####################

structure_img = IMG[0, 0, structure_channel,:,:,:]
view(single_fluorescent_view(structure_img))

## Image segmentation

### Step 1: Pre-Processing

The Allen Cell Segmenter team recommends using Min-Max normalization for dot-like structures. They include an "upper-bound" parameter, which should be set to a level just above the brightest "real" pixels in the image. Anything above this upper bound will be reset to zero, which is meant to eliminate super bright pixels that are imaging artifacts. For our data, we don't use this upper bound, but you may wish to change this parameter, depending on the intensity profile of your images.  


* **Smoothing** 

Here, we use 2D gaussian smoothing with `gaussian_smoothing_sigma = 1`. If your data is very noisy, you may find increasing `gaussian_smoothing_sigma` from 1 to a higher value, like 1.5 or 2 beneficial. We use the default slice-by-slice gaussian smoothing, but the 3D smoothing process may be useful. You can try `image_smoothing_gaussian_3d(struct_img, sigma=gaussian_smoothing_sigma)` with `gaussian_smoothing_sigma = 1`.


In [ ]:
################################
intensity_scaling_param = [0]
gaussian_smoothing_sigma = 1
################################

# intensity normalization
structure_img = intensity_normalization(structure_img, scaling_param=intensity_scaling_param)

# smoothing with gaussian filter
structure_img_smooth = image_smoothing_gaussian_slice_by_slice(structure_img, sigma=gaussian_smoothing_sigma)

In [ ]:
# quickly visualize the image after smoothing
view(single_fluorescent_view(structure_img_smooth))

### Step 2: Core Algorithm

#### step 2.1: Apply 3D Spot filter (S3)

Parameter syntax: `[[scale_1, cutoff_1], [scale_2, cutoff_2], ....]` 
* `scale_x` is set based on the estimated radius of your target dots. For example, if visually the diameter of the dots is usually 3~4 pixels, then you may want to set `scale_x` as `1` or something near `1` (like `1.25`). Multiple scales can be used, if you have dots of very different sizes.  
* `cutoff_x` is a threshold applied on the actual filter reponse to get the binary result. Smaller `cutoff_x` may yielf more dots and fatter segmentation, while larger `cutoff_x` could be less permisive and yield less dots and slimmer segmentation. 


In [ ]:
################################
## PARAMETERS for this step ##
s3_param = [[1, 0.02]]
################################

bw = dot_3d_wrapper(structure_img_smooth, s3_param)

In [ ]:
# view the segmentation result
viewer_bw = view(segmentation_quick_view(bw))
viewer_bw

##### After quickly visualizing the segmentation results, you can also visualize the segmentation and original image side by side
##### You may select an ROI to inspect the details

* Option 1: Easy ROI selection, but NOT recommended if you are using a laptop

You can select an ROI in above visualization ('viewer_bw'); otherwise, the default ROI is the full image

[See this video for How to select ROI](https://www.youtube.com/watch?v=ZO8ey6-tF_0&index=3&list=PL2lHcsoU0YJsh6f8j2vbhg2eEpUnKEWcl)

* Option 2: Manually type in ROI coordinates

Type in the coordinates of upper left corner and lower right corner of the ROI in the form of [Upper_Left_X, Upper_Left_Y, Lower_right_X, Lower_right_Y]. 

In [ ]:
# Option 1:
# view(seg_fluo_side_by_side(structure_img,bw,roi=['ROI',viewer_bw.roi_slice()]))

# Option 2: 
view(seg_fluo_side_by_side(structure_img,bw,roi=['M',[570,370,730,440]]))

##### Is the segmentation satisfactory? Here are some possible criteria:
* Is there any spot should be detected but not? Try to reduce `cutoff_x`
* Is there any spot should not be detected but actually appear in the result? Try to increase `cutoff_x` or try a larger `scale_x`
* Is the segmented size of the spots fatter than it should be? Try to increase `cutoff_x` or try a smaller `scale_x`
* Is there any spot that should be solid but segmented as a ring? Try to increase `scale_x`
* Are you observing spots with very different sizes? Try multiple sets of `scale_x` and `cutoff_x` 


#### If the results are satisfactory, go to Step 3; otherwise, try to tweak the parameters based on the above suggestions. 

Assumption: the segmentation result is saved in a variable named `bw`.

#### Step 3: Post-Processing 

In [ ]:
################################
## PARAMETERS for this step ##
minArea = 200
################################

final_seg = remove_small_objects(bw>0, min_size=minArea, connectivity=1, in_place=False)

## Result inspection

In [ ]:
viewer_final = view(segmentation_quick_view(final_seg))
viewer_final

### You can also focus your inspection on a small ROI

* Option 1: Easy ROI selection, but NOT recommended if you are using a laptop

You can select an ROI in above visualization ('viewer_final'); otherwise, the default ROI is the full image

[See this video for How to select ROI](https://www.youtube.com/watch?v=ZO8ey6-tF_0&index=3&list=PL2lHcsoU0YJsh6f8j2vbhg2eEpUnKEWcl)

* Option 2: Manually type in ROI coordinates

Type in the coordinates of upper left corner and lower right corner of the ROI in the form of [Upper_Left_X, Upper_Left_Y, Lower_right_X, Lower_right_Y]. 

In [ ]:
# Option 1: 
# view(seg_fluo_side_by_side(structure_img, final_seg, roi=['ROI',viewer_final.roi_slice()]))

# Option 2: 
view(seg_fluo_side_by_side(structure_img, final_seg, roi=['M',[267,474, 468, 605]]))

### You can also inspect the same ROI on the effect of final cutting step (the same ROI as above)

In [ ]:
# Option 1 for ROI selection:
# view(seg_fluo_side_by_side(bw, final_seg, roi=['ROI',viewer_final.roi_slice()]))

# Option 2 for ROI selection:
view(seg_fluo_side_by_side(bw, final_seg, roi=['M',[570,370,730,440]]))
# left is the final version
# right is the version before cutting

### You may also physically save the segmentation results into a .tiff image

In [ ]:
# define where to save your test segmentations

output_filepath = '/output/test-segmentations/'

if not os.path.isdir(output_filepath):
    os.makedirs(output_filepath)


In [ ]:
# this file will be saved within your docker container volume "output"
# in order to visualize this most easily, you can copy this to your computer using
# docker cp jupyter:/output/ output/ 

output_seg = final_seg>0
out=output_seg.astype(np.uint8)
out[out>0]=255
imageio.volwrite(output_filepath + FILE_NAME + '-test_cent_seg.tiff', out)
